In [4]:
import pandas as pd
import numpy as np
import matplotlib as plt


data = pd.read_csv('data.csv', parse_dates=True)
data['candle_time'] = pd.to_datetime(data['candle_time'], format="%d-%m-%Y %H:%M", dayfirst=True) 
data.set_index('candle_time', inplace=True)



data_m = data.diff()
data_m = data_m.dropna()


ratios = {
    'Monday': {'SB6': 2, 'SB4v2': 3, 'DNIv5': 2, 'DNIv6': 3},
    'Tuesday': {'SB6': 4, 'SB4v2': 2, 'DNIv5': 2, 'DNIv6': 2},
    'Wednesday': {'SB6': 1, 'SB4v2': 4, 'DNIv5': 1, 'DNIv6': 3},
    'Thursday': {'SB6': 3, 'SB4v2': 4, 'DNIv5': 1, 'DNIv6': 2},
    'Friday': {'SB6': 3, 'SB4v2': 4, 'DNIv5': 2, 'DNIv6': 1},
}

ratios_ewm = {
    'Monday': {'SB6': 2.5, 'SB4v2': 2.5, 'DNIv5': 2.5, 'DNIv6': 2.5},
    'Tuesday': {'SB6': 2.5, 'SB4v2': 2.5, 'DNIv5': 2.5, 'DNIv6': 2.5},
    'Wednesday': {'SB6': 2.5, 'SB4v2': 2.5, 'DNIv5': 2.5, 'DNIv6': 2.5},
    'Thursday': {'SB6': 2.5, 'SB4v2': 2.5, 'DNIv5': 2.5, 'DNIv6': 2.5},
    'Friday': {'SB6': 2.5, 'SB4v2': 2.5, 'DNIv5': 2.5, 'DNIv6': 2.5},
}

def calculate_portfolio(row):
    day = row.name.day_name()  # Access the day name from the index
    ratio = ratios.get(day, {'SB6': 1, 'SB4v2': 1, 'DNIv5': 1, 'DNIv6': 1})  # Default ratio if day is not found
    portfolio_value = (row['SB6'] * ratio['SB6'] +
                       row['SB4v2'] * ratio['SB4v2'] +
                       row['DNIv5'] * ratio['DNIv5'] +
                       row['DNIv6'] * ratio['DNIv6'])
    return portfolio_value

def calculate_portfolio_ewm(row):
    day = row.name.day_name()  # Access the day name from the index
    ratio = ratios_ewm.get(day, {'SB6': 1, 'SB4v2': 1, 'DNIv5': 1, 'DNIv6': 1})  # Default ratio if day is not found
    portfolio_value = (row['SB6'] * ratio['SB6'] +
                       row['SB4v2'] * ratio['SB4v2'] +
                       row['DNIv5'] * ratio['DNIv5'] +
                       row['DNIv6'] * ratio['DNIv6'])
    return portfolio_value

data_m['portfolio'] = data_m.apply(calculate_portfolio, axis=1)
data_m['portfolio_ewm'] = data_m.apply(calculate_portfolio_ewm, axis=1)


base = 8000000
portfolio_df = pd.DataFrame()
portfolio_df['portfolio'] = data_m['portfolio']/base
portfolio_df['portfolio_ewm'] = data_m['portfolio_ewm']/base

portfolio_df['date'] = portfolio_df.index.date
daily_returns = portfolio_df.groupby('date').sum()

# Reset index to make 'date' a column
daily_returns.reset_index(inplace=True)

daily_returns.set_index('date', inplace=True)
daily_returns

,portfolio,portfolio_ewm
date,,
2021-01-04,-0.003094,-0.002845
2021-01-05,-0.000558,-0.000148
2021-01-06,0.002160,0.001477
2021-01-07,0.004153,0.002972
2021-01-08,-0.002879,-0.002523
...,...,...
2024-07-01,0.000689,0.000682
2024-07-02,0.000740,0.000578
2024-07-03,0.000342,0.000300
